In [1]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from linodenet.models import ResNet, ResNetBlock
from linodenet.util import initialize_from_config
from torchinfo import summary

In [3]:
model = ResNetBlock(input_size=4)
summary(model, (1, 4))

In [8]:
model = ResNet(input_size=128)
summary(model)

In [ ]:
import importlib
module = importlib.import_module("torch.nn")

In [5]:
module = __import__("torch.nn")

In [3]:
ResNet(input_size=4)

In [4]:
.HP

In [3]:
ResNet(input_size=4)

In [7]:
d.pop("key")

In [8]:
HP: dict = {
    "input_size" : None,
    "num_blocks": 2,
    "layers": [
        {
            "__name__": "BatchNorm1d",
            "__source__": "torch.nn",
            "num_features": None,
            "eps": 1e-05,
            "momentum": 0.1,
            "affine": True,
            "track_running_stats": True,
        },
        {
            "__name__": "ReLU",
            "__source__": "torch.nn",
            "inplace": False,
        },
        {
            "__name__": "Linear",
            "__source__": "torch.nn",
            "in_features": None,
            "out_features": None,
            "bias": True,
        },
    ],
}

In [10]:
class A:
    class B:
        print(locals())